In [24]:
import pandas as pd
import numpy as np

In [25]:
df_full = pd.read_pickle('../../Data/block4_concat_lane.pkl')  \
    .set_index('edge_id', append=True) \
    .reorder_levels((0,1,3,2))

df_full.head()

lat        lon    speed  \
file_name id edge_id             time                                   
4_1       1  250699362_250699984 42.00  37.982746  23.732961  11.9046   
                                 42.04  37.982746  23.732963  11.8975   
                                 42.08  37.982747  23.732964  11.8919   
                                 42.12  37.982748  23.732965  11.8871   
                                 42.16  37.982748  23.732966  11.8831   

                                        lon_acc  lat_acc  type  traveled_d  \
file_name id edge_id             time                                        
4_1       1  250699362_250699984 42.00  -0.1145   0.0138  Taxi      182.37   
                                 42.04  -0.1007   0.0147  Taxi      182.37   
                                 42.08  -0.0918   0.0157  Taxi      182.37   
                                 42.12  -0.0869   0.0167  Taxi      182.37   
                                 42.16  -0.0784   0.0176  Taxi      182.37   

                                        avg_speed   bearing  \
file_name id edge_id             time                         
4_1       1  250699362_250699984 42.00   9.740748  1.570795   
                                 42.04   9.740748  0.168572   
                                 42.08   9.740748  0.168573   
                                 42.12   9.740748  1.570796   
                                 42.16   9.740748  0.328080   

                                        nearest_edge_start_node  ...  dir  \
file_name id edge_id             time                            ...        
4_1       1  250699362_250699984 42.00                250699362  ...    0   
                                 42.04                250699362  ...    0   
                                 42.08                250699362  ...    0   
                                 42.12                250699362  ...    0   
                                 42.16                250699362  ...    0   

                                        xtrack_dist  time_stamp  \
file_name id edge_id             time                             
4_1       1  250699362_250699984 42.00    -1.883401       42.00   
                                 42.04    -1.980795       42.04   
                                 42.08    -1.937041       42.08   
                                 42.12    -1.893287       42.12   
                                 42.16    -1.941984       42.16   

                                        edge_progress_intervals     len  \
file_name id edge_id             time                                     
4_1       1  250699362_250699984 42.00                      0.3  97.581   
                                 42.04                      0.3  97.581   
                                 42.08                      0.3  97.581   
                                 42.12                      0.3  97.581   
                                 42.16                      0.3  97.581   

                                        lanes  node_veh_dist  edge_seg  \
file_name id edge_id             time                                    
4_1       1  250699362_250699984 42.00    5.4      29.814330       1.0   
                                 42.04    5.4      29.674830       1.0   
                                 42.08    5.4      29.537753       1.0   
                                 42.12    5.4      29.400718       1.0   
                                 42.16    5.4      29.330986       1.0   

                                        vehicle_density  avg_surr_speed  
file_name id edge_id             time                                    
4_1       1  250699362_250699984 42.00                7       10.464171  
                                 42.04                7       10.457843  
                                 42.08                7       10.452857  
                                 42.12                7       10.448586  
                                 42.16        

In [26]:
df_full['xtrack_diff'] = df_full.xtrack_dist \
    .groupby(df_full.index.names[-1]) \
    .apply(lambda x: (x - x.shift(-1)).fillna(0))
    
df_full.xtrack_diff

file_name  id    edge_id               time  
4_1        1     250699362_250699984   42.00    -4.667009
                                       42.04    -4.764403
                                       42.08    -4.720649
                                       42.12    -4.676895
                                       42.16    -4.725592
                                                   ...   
1_3        2116  300400248_8446047162  921.64    0.000000
                                       921.68    0.000000
                                       921.72    0.000000
                                       921.76    0.000000
                                       921.80    0.000000
Name: xtrack_diff, Length: 2997687, dtype: float64

In [27]:
 # helper functions

def speed_ratio(grp, min_speed=0):
    return len(grp[grp.speed > min_speed]) / len(grp)

def pivot(A, col_factor):
    c = A.shape[1]
    if A.size < col_factor*c:
        return None
    r_new = A.size // (col_factor*c)
    A = A[:col_factor*r_new]
    return A.to_numpy().reshape(r_new, col_factor*c)

In [30]:
# filtering thresholds
min_traj_len = 300
min_speed_ratio = 0.75

# filter out unwanted trajectories
df = df_full.groupby(df_full.index.names[:-1]) \
            .filter(lambda grp: (len(grp) >= min_traj_len) & (speed_ratio(grp) >= min_speed_ratio))


# features to aggregate (only using mean)
feature_list = ['xtrack_diff','speed','vehicle_density','lat_acc','lon_acc']


# factor by which the number of columns will increase after pivoting
col_factor = 4


# size of aggregating window in seconds
window = '5S'


# function for groupby.apply that will downsample a trajectory and pivot it
def f(grp):
    grp = grp.reset_index(level=(0,1,2), drop=True)
    grp.index = pd.TimedeltaIndex(grp.index,unit='s')
    grp = grp.resample(window).mean().reset_index(drop=True)
    return pd.DataFrame(pivot(grp,col_factor))


df = df[feature_list] \
    .groupby(df[feature_list].index.names[:-1]) \
    .apply(f) \
    .dropna() \
    .reset_index(level=-1, drop=True)

df.columns = [feature+'_'+str(i) for i in range(col_factor) for feature in feature_list]
df

xtrack_diff_0    speed_0  \
file_name id   edge_id                                         
1_2       6    250691795_250699359      -0.742157  21.733537   
          14   250691795_250699359       3.440826  26.956150   
          17   250691795_250699359       2.236424  28.887475   
          20   250691795_250699359       2.969167  26.966758   
          22   250691795_250699359      -0.168996  30.671610   
...                                           ...        ...   
4_3       1653 250699362_250699984      -2.915260  16.722614   
          1657 250699362_250699984      -1.934443  16.543075   
          1663 250699614_250699615      -7.025841  11.529435   
          1678 250699362_250699984      -2.802854  22.982110   
          1686 250699362_250699984       0.471425  24.623982   

                                    vehicle_density_0  lat_acc_0  lon_acc_0  \
file_name id   edge_id                                                        
1_2       6    250691795_250699359              2.160   0.979492   0.482154   
          14   250691795_250699359              3.088   0.053814   0.103854   
          17   250691795_250699359              4.216   0.072912  -0.455759   
          20   250691795_250699359              3.784   0.044249  -0.386796   
          22   250691795_250699359              2.296   0.037418  -0.454366   
...                                               ...        ...        ...   
4_3       1653 250699362_250699984              1.360  -0.049068  -0.106043   
          1657 250699362_250699984              1.208  -0.056801  -0.231152   
          1663 250699614_250699615              1.520   0.120728  -0.795630   
          1678 250699362_250699984              1.552  -0.031931  -0.079619   
          1686 250699362_250699984              2.312  -0.010266  -0.073733   

                                    xtrack_diff_1    speed_1  \
file_name id   edge_id                                         
1_2       6    250691795_250699359       2.797466  22.738274   
          14   250691795_250699359       5.267725  20.737234   
          17   250691795_250699359       0.904362  21.876234   
          20   250691795_250699359       2.582236  24.561676   
          22   250691795_250699359      -0.352559  24.280664   
...                                           ...        ...   
4_3       1653 250699362_250699984       0.057214  14.832430   
          1657 250699362_250699984       3.345107  13.138346   
          1663 250699614_250699615      -3.703711   0.897658   
          1678 250699362_250699984      -2.982493  19.093694   
          1686 250699362_250699984       1.273754  19.062535   

                                    vehicle_density_1  lat_acc_1  lon_acc_1  \
file_name id   edge_id                                                        
1_2       6    250691795_250699359              1.992   0.069279  -0.366894   
          14   250691795_250699359              3.872   0.002274  -0.311239   
          17   250691795_250699359              3.824  -0.028101  -0.090294   
          20   250691795_250699359              4.192   0.013367   0.127663   
          22   250691795_250699359              3.560   0.008998  -0.137922   
...                                               ...        ...        ...   
4_3       1653 250699362_250699984              1.400   0.049378   0.008786   
          1657 250699362_250699984              1.288   0.032078  -0.119600   
          1663 250699614_250699615              1.352  -0.003289  -0.344867   
          1678 250699362_250699984              2.096  -0.003070  -0.211122   
          1686 250699362_250699984              2.800  -0.014959  -0.430257   

                                    xtrack_diff_2    speed_2  \
file_name id   edge_id                                         
1_2       6    250691795_250699359       4.436867   9.737934   
          14   250691795_250699359       6.389214  20.285573   
          17   250691795_250699359       1.060061  21.0344

In [23]:
# example trajectory
df.loc[('1_2',   6,  '250691795_250699359')]

xtrack_diff_0    speed_0  vehicle_density_0  \
file_name id edge_id                                                            
1_2       6  250691795_250699359      -0.742157  21.733537              2.160   
             250691795_250699359       4.436867   9.737934              3.904   
             250691795_250699359       2.895003   8.895566              4.416   

                                  lat_acc_0  lon_acc_0  xtrack_diff_1  \
file_name id edge_id                                                    
1_2       6  250691795_250699359   0.979492   0.482154       2.797466   
             250691795_250699359   0.034467  -0.994580       3.291725   
             250691795_250699359   0.038436   1.031973       0.877818   

                                    speed_1  vehicle_density_1  lat_acc_1  \
file_name id edge_id                                                        
1_2       6  250691795_250699359  22.738274              1.992   0.069279   
             250691795_250699359   0.038267              4.800  -0.000918   
             250691795_250699359  20.324398              4.000   0.029646   

                                  lon_acc_1  
file_name id edge_id                         
1_2       6  250691795_250699359  -0.366894  
             250691795_250699359  -0.039926  
             250691795_250699359   0.391052